In [2]:
# setup
import pandas as pd
import numpy as np
import random
import statsmodels.api as sm
import sys
from ppi_py import ppi_ols_ci, classical_ols_ci, ppi_ols_pointestimate
import matplotlib.pyplot as plt

In [3]:
rho_intervention = 0.35
n0 = 1000
ninf = (n0*(1-rho_intervention**2)*(1+0.01)).astype(int) 
print(ninf)
ns = np.arange(ninf, n0, 5)
Ns = ns*(n0 - ns)/(ns - n0*(1 - rho_intervention**2))
Ns = Ns.astype(int)
reps = 100
ppi_ses = np.zeros((len(ns),reps))

x = "Intervention"	
y = "gpt4turbo_wp_Saved"

for i, n in enumerate(ns):
    N = Ns[i]
    print(f"n: {n}, N: {N}")
    for r in range(reps):
        # sample dilemmas for human subjects sample
        n_ids = random.sample(sorted(ids), k=n)
        
        # get remaining dilemma ids to sample from
        remaining_ids = list(set(ids) - set(n_ids))

        # skip current iteration if target N is larger than population
        if (len(remaining_ids) < N):
            continue 
        
        # sample dilemmas for silicon subjects sample
        N_ids = random.sample(remaining_ids, k=N)

        # subset data
        df_human = df[ df["ResponseID"].isin(n_ids) ]
        df_silicon = df [ df["ResponseID"].isin(N_ids)]

        # pool human and silicon subject decisions
        df_pooled = pd.concat([df_human,df_silicon], ignore_index=True)

        # compute ppi acme
        ppi = compute_amce_ppi(n_data=df_human, N_data=df_silicon, x=x, y=y)

        ppi_ses[i,r] = (ppi["conf_high_ppi"][0] - ppi["conf_low_ppi"][0])
    print(ppi_ses[i].mean())

AttributeError: 'float' object has no attribute 'astype'

In [ ]:

gamma = 1/100
n_star = n0*(1-rho_intervention**2 + np.sqrt(gamma*rho_intervention**2*(1- rho_intervention**2)))
n_star = n_star.astype(int)
c_star = (1 - rho_intervention**2 + gamma*rho_intervention**2 + 2*np.sqrt(gamma*rho_intervention**2*(1- rho_intervention**2)))



cost = (ns + gamma*(ns + Ns))/n0

plt.plot(ns, ppi_ses.mean(axis = 1)*10, label="PPI SE")
sd = 2*ppi_ses.std(axis=1)*10/np.sqrt(reps)
plt.fill_between(ns, ppi_ses.mean(axis=1)*10 - sd, ppi_ses.mean(axis=1)*10 + sd, alpha=0.2)
plt.plot(ns, cost, label="Cost")
plt.axvline(x=n_star, color='r', linestyle='--', label="n*")
plt.title("The cheapest pair achieving a given PPI SE")
plt.xlabel("n")
plt.ylabel("PPI SE / Cost")
plt.legend()

In [ ]:
rho_intervention = results2["rho"][0]
B = 1000
gamma = 1/100
n0 = B/(1 -  rho_intervention**2 + gamma*rho_intervention**2 + 2*np.sqrt(gamma*rho_intervention**2*(1- rho_intervention**2)))
nstar = n0*(1-rho_intervention**2 + np.sqrt(gamma*rho_intervention**2*(1- rho_intervention**2)))
nstar = nstar.astype(int)
print(nstar)



ns = np.arange(nstar - (B - nstar), B+5, 5)
ns = ns.astype(int)
Ns = (B - (1+gamma)*ns)/gamma
Ns = Ns.astype(int)
ns = ns[Ns > 0]
Ns = Ns[Ns > 0]
reps = 100
ppi_ses2 = np.zeros((len(ns),reps))

x = "Intervention"	
y = "gpt4turbo_wp_Saved"

for i, n in enumerate(ns):
    N = Ns[i]
    print(f"n: {n}, N: {N}")
    for r in range(reps):
        # sample dilemmas for human subjects sample
        n_ids = random.sample(sorted(ids), k=n)
        
        # get remaining dilemma ids to sample from
        remaining_ids = list(set(ids) - set(n_ids))

        # skip current iteration if target N is larger than population
        if (len(remaining_ids) < N):
            continue 
        
        # sample dilemmas for silicon subjects sample
        N_ids = random.sample(remaining_ids, k=N)

        # subset data
        df_human = df[ df["ResponseID"].isin(n_ids) ]
        df_silicon = df [ df["ResponseID"].isin(N_ids)]

        # pool human and silicon subject decisions
        df_pooled = pd.concat([df_human,df_silicon], ignore_index=True)

        # compute ppi acme
        ppi = compute_amce_ppi(n_data=df_human, N_data=df_silicon, x=x, y=y)

        ppi_ses2[i,r] = (ppi["conf_high_ppi"][0] - ppi["conf_low_ppi"][0])
    print(ppi_ses2[i].mean())

In [ ]:
cost = (ns + gamma*(ns + Ns))/B

plt.plot(ns, ppi_ses2.mean(axis = 1)*10, label="PPI SE")
sd = 2*ppi_ses2.std(axis=1)*10/np.sqrt(reps)
plt.fill_between(ns, ppi_ses2.mean(axis=1)*10 - sd, ppi_ses2.mean(axis=1)*10 + sd, 
                alpha=0.2)
plt.plot(ns, cost, label="Cost")
plt.axvline(x=nstar, color='r', linestyle='--', label="n*")
plt.title("The most powerful pair within a given budget")
plt.xlabel("n")
plt.ylabel("PPI SE / Cost")
plt.legend()